# Start Up: 
Before Running the cell below, you must ensure that these have been run in Terminal **IN ORDER** : 
- conda update -n base -c defaults conda 

    - cd SageMaker
    
      - cd yelp-dataset-challenge-1-ds
      
         - conda env create -f environment.yml
          
            - source activate ydc1 
                
                - pip install python-decouple
                  
                  - pip install pprintpp
                  
# Spacy Installs: 

   - python -m spacy download en_core_web_lg

        - python -m spacy link en_core_web_lg en

In [1]:
import json

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import spacy
#import s3
#from pprintpp import pprint as pp
from sklearn.externals import joblib
nlp = spacy.load('en')

# Load in Bucket
#bucket = s3.Bucket('yelpchallenge1')
# Look inside the bucket.
#bucket.contents

In [ ]:
                    ### ***** DO NOT RUN. ******* #### 
                  ### ***** ALREADY INSTALLED. ****** ###

# Only have to run this once.
# Installs the .csv 'Locally' on SageMaker Instance

#bucket.get('datasets/df.csv', 'df.csv')

# Installing user.json 'Locally'
#bucket.get('datasets/user.json', 'user.json')


    # Load in Bucket
# bucket = s3.Bucket('yelpchallenge1')
    # Look inside the bucket.
# bucket.contents

# Getting Started: Imports

In [4]:
    # Read-in final.csv
final_df = pd.read_csv('final_df.csv')

    # Read-in df.csv
#df = pd.read_csv('df.csv', nrows=250000)


    # Read-in review_df.csv
#review_df = pd.read_csv('review.csv')

    # Read-in dtm_final.csv (FINAL)
dtm_final = pd.read_csv('dtm_final.csv')

    # import Vectorizer Model
#vect2 = joblib.load('vect_2.sav')

In [5]:
dtm_final.head(5)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,-PRON-,-PRON-,-PRON- -PRON-,-PRON- a,-PRON- all,-PRON- also,-PRON- and,...,worth,would,would be,would have,would not,wrong,year,yelp,yet,stars
0,0,0,0,0.069829,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,1.0
1,1,1,1,0.343331,0.0,0.000000,0.0,0.0,0.0,0.047998,...,0.052035,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,5.0
2,2,2,2,0.314034,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.093163,0.0,0.000000,5.0
3,3,3,3,0.080641,0.0,0.000000,0.0,0.0,0.0,0.000000,...,0.000000,0.087372,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000,5.0
4,4,4,4,0.515313,0.0,0.020392,0.0,0.0,0.0,0.043038,...,0.000000,0.029004,0.0,0.0,0.02601,0.027762,0.021839,0.0,0.028812,1.0


In [6]:
dtm_final = dtm_final.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [7]:
dtm_final = dtm_final.drop(columns = ['stars'])

In [9]:
dtm_final.head()

,-PRON-,-PRON-,-PRON- -PRON-,-PRON- a,-PRON- all,-PRON- also,-PRON- and,-PRON- be,-PRON- but,-PRON- can,...,work,worth,would,would be,would have,would not,wrong,year,yelp,yet
0,0.069829,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000
1,0.343331,0.0,0.000000,0.0,0.0,0.0,0.047998,0.122973,0.000000,0.000000,...,0.000000,0.052035,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000
2,0.314034,0.0,0.000000,0.0,0.0,0.0,0.000000,0.039198,0.000000,0.086683,...,0.084567,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.093163,0.0,0.000000
3,0.080641,0.0,0.000000,0.0,0.0,0.0,0.000000,0.110721,0.000000,0.000000,...,0.000000,0.000000,0.087372,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000
4,0.515313,0.0,0.020392,0.0,0.0,0.0,0.043038,0.027566,0.028836,0.020320,...,0.000000,0.000000,0.029004,0.0,0.0,0.02601,0.027762,0.021839,0.0,0.028812


In [10]:
dtm_final.shape

(95000, 773)

In [11]:
final_df.columns

Index(['Unnamed: 0', 'useful', 'funny', 'cool', 'text', 'tokens'], dtype='object')

In [ ]:
# Dropping All Rows with 0 in Interactions Column 
final_df = final_df[final_df.useful !=0]
final_df = final_df[final_df.funny !=0]
final_df = final_df[final_df.cool !=0]

In [12]:
final_df.head(5)

,Unnamed: 0,useful,funny,cool,text,tokens
0,0,5.0,4.0,5,Tracy dessert had a big name in Hong Kong and ...,"['tracy', 'dessert', 'hong', 'kong', 'markham'..."
1,1,3.0,1.0,1,This place has gone down hill. Clearly they h...,"['place', 'gone', 'hill', 'clearly', 'staff', ..."
2,2,1.0,7.0,1,I love chinese food and I love mexican food. W...,"['love', 'chinese', 'food', 'love', 'mexican',..."
3,3,9.0,6.0,9,If you are looking for the best pierogies in P...,"['looking', 'best', 'pierogies', 'pittsburgh',..."
4,4,7.0,1.0,1,"Unfortunately, I must recommend not to conduct...","['unfortunately', 'recommend', 'conduct', 'bus..."


In [ ]:
print(final_df['useful'].dtype)
print(final_df['funny'].dtype)
print(final_df['cool'].dtype)

In [ ]:
# Converting columns with Dtypes Float64 to Int64 
final_df['useful'] = final_df['useful'].astype('Float64')
final_df['funny'] = final_df['funny'].astype('Float64')

In [ ]:
print(final_df['useful'].dtype)
print(final_df['funny'].dtype)

In [ ]:
final_df.head(5)

In [13]:
final_df = final_df.drop(columns = ['Unnamed: 0'])

In [14]:
final_df.head(1)

,useful,funny,cool,text,tokens
0,5.0,4.0,5,Tracy dessert had a big name in Hong Kong and ...,"['tracy', 'dessert', 'hong', 'kong', 'markham'..."


In [ ]:
final_df.to_csv(index=True)
final_df.to_csv(r'final_df.csv')

In [15]:
dtm = dtm_final.iloc[0:10000]

In [16]:
dtm.head(5)

,-PRON-,-PRON-,-PRON- -PRON-,-PRON- a,-PRON- all,-PRON- also,-PRON- and,-PRON- be,-PRON- but,-PRON- can,...,work,worth,would,would be,would have,would not,wrong,year,yelp,yet
0,0.069829,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000
1,0.343331,0.0,0.000000,0.0,0.0,0.0,0.047998,0.122973,0.000000,0.000000,...,0.000000,0.052035,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000
2,0.314034,0.0,0.000000,0.0,0.0,0.0,0.000000,0.039198,0.000000,0.086683,...,0.084567,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.093163,0.0,0.000000
3,0.080641,0.0,0.000000,0.0,0.0,0.0,0.000000,0.110721,0.000000,0.000000,...,0.000000,0.000000,0.087372,0.0,0.0,0.00000,0.000000,0.000000,0.0,0.000000
4,0.515313,0.0,0.020392,0.0,0.0,0.0,0.043038,0.027566,0.028836,0.020320,...,0.000000,0.000000,0.029004,0.0,0.0,0.02601,0.027762,0.021839,0.0,0.028812


# Model Prep: 

**Description**: 

Combining based on their *Unique Account ID's.*
The end Product will be One DataFrame Consisting of Each Account:
- **Name**, 
- **User_ID**,
- **Review_ID**,
- **Text**,
- **That Users respective review(s)**,
- **Interactions that Review (i.e: Cool, Funny, Useful)**  

The goal of the model is to have the ability to type in the Review you are wanting to post on Yelp, and give the User the ability to Predict What type of Interaction they would potentially receive and Total Number of each interaction. The model Accuracy will be Displayed beside the Prediction. 

# Random Forest Classifier: Useful

In [40]:
# Imports: 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier, SGDRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
# Defining
# tokenizer
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))

In [20]:
# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=15, algorithm='ball_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=15, p=2, radius=1.0)

In [22]:
# Creating Fake Review for Testing: 
fake_review = """
After finshing our dinner we heard a loud crash come from the back of the place The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation given that I was completley drunk and could not stop laughing. The beer was a little flat, the 'chicken' was a little gamey. 
Service was great 4 Stars.
"""

# tokenizer
vect_review = tfidf.fit_transform(tokenize(fake_review)

In [27]:
# Splitting Dataset into training sets

    # Using 10,000 Rows. 

reviews = final_df['tokens'].iloc[0:25000]
interactions = final_df['useful'].iloc[0:25000]

In [28]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()


# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])


#Tuning
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, 0.05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs= -1, verbose=1)
grid_search.fit(reviews, interactions)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


/Users/haden/anaconda3/envs/ydc1/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   53.2s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  3.1min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (500, 1000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [29]:
grid_search.best_score_

0.24655577956989247

In [30]:
grid_search.best_params_

{'clf__max_depth': 15,
 'clf__n_estimators': 10,
 'vect__max_df': 0.75,
 'vect__max_features': 500,
 'vect__min_df': 0.02}

In [31]:
# Fake Review Interaction 'Useful' Prediction: 

useful_pred = grid_search.predict([r'vect_review'])
print(useful_pred)

[1.]


In [ ]:
#Pickling Model with JobLib
joblib.dump(useful_pred, 'useful_clf.joblib')

In [ ]:
# Creating Fake Review for Testing: 
fake_review4 = """
After finshing our dinner we heard a loud crash come from the back of the place The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation given that I was completley drunk and could not stop laughing. The beer was a little flat, the 'chicken' was a little gamey. 
Service was great 4 Stars.
"""

In [ ]:
# tokenizing Fake_Review4
vect_review4 = tfidf.fit_transform(tokenize(fake_review4))

In [ ]:
useful_pred4 = grid_search.predict([r'vect_review4'])
print(useful_pred4)

In [ ]:
# tokenizer
vect_review2 = tfidf.fit_transform(tokenize(fake_review2))

In [ ]:
useful_pred2 = grid_search.predict([r'vect_review2'])
print(useful_pred2)

In [ ]:
final_df['text'].iloc[7]

In [ ]:
final_df.iloc[4]

In [ ]:
# Dropped all 0's from Rows 
grid_search.best_score_

In [ ]:
grid_search.best_params_

In [ ]:
# Creating Fake Review5 for Testing: 
fake_review5 = """
After finshing our dinner we heard a loud crash come from the back of the place The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation given that I was completley drunk and could not stop laughing. The beer was a little flat, the 'chicken' was a little gamey. 
Service was great 4 Stars.
"""

In [ ]:
# tokenizing Fake_Review5
vect_review5 = tfidf.fit_transform(tokenize(fake_review5))

In [ ]:
# Fake Review Interaction 'Useful' Prediction: 

useful_pred5 = grid_search.predict([r'vect_review5'])
print(useful_pred5)

In [ ]:
# Still getting 0...... 

In [32]:
# Testing with a Randomly pulled Review from Final_df for tokenization and prediction. 
review_888 = final_df['text'].iloc[888]
# Interaction 'Useful' Prediction: 
vect_review6 = tfidf.fit_transform(tokenize(review_888))
useful_pred6 = grid_search.predict([r'vect_review6'])
print(useful_pred6)

[1.]


# Random Forest Classifier for 'Funny' after Changes: as of 1/09/2020 at 7:00PM EST

In [ ]:
# Imports: 
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Creating Variables for Training 
interactions = final_df['funny']

# Splitting Dataset into training sets

    # Using 10,000 Rows. 

reviews = final_df['text'].iloc[0:10000]
interactions = interactions.iloc[0:10000]

# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nn.fit(dtm)

# Defining
# tokenizer
def tokenize(document):
    doc = nlp(document)
    return [token.lemma_ for token in doc if (token.is_stop != True) and (token.is_punct != True)]

tfidf = TfidfVectorizer(tokenizer=tokenize, min_df=0.025, max_df=.95, ngram_range=(1,2))


# Creating Fake Review for Testing: 
fake_review = """
After finshing our dinner we heard a loud crash come from the back of the place The next thing we saw was a cook running out of the kitchen chasing a cat with a cleaver covered in blood and animal hair... My children were traumatized.. Given that the cat looked exactly like Whiskers, who ran away last week. 
I had no idea how I was going to parent that situation given that I was completley drunk and could not stop laughing. The beer was a little flat, the 'chicken' was a little gamey. 
Service was great 4 Stars.
"""

In [ ]:
# tokenizer
vect_review1 = tfidf.fit_transform(tokenize(fake_review))

In [ ]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()


# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect), 
                 # Classifier
                 ('clf', rfc)
                ])


#Tuning
parameters = {
    'vect__max_df': (0.75, 1.0),
    'vect__min_df': (.02, 0.05),
    'vect__max_features': (500,1000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs= -1, verbose=1)
grid_search.fit(reviews, interactions)

In [ ]:
# Score
print(grid_search.best_score_) 
print(grid_search.best_params_)

In [ ]:
# Fake Review Interaction 'Cool' Prediction: 

cool_pred = grid_search.predict([r'vect_review1'])
print(cool_pred)

In [ ]:
#Pickling Model with JobLib
from sklearn.externals import joblib
joblib.dump(regressor, 'regressor.joblib')

# SGDRegressor Model: 'Useful' Prediction

In [70]:
tfidf = TfidfVectorizer(stop_words='english')
scaler = StandardScaler(with_mean=False, with_std=False)
sgdc = SGDRegressor(alpha=0.0001, 
                    average=False, 
                    early_stopping=False,
                    epsilon=0.1,
                    eta0=0.01, 
                    fit_intercept=True, 
                    l1_ratio=0.15,
                    learning_rate='adaptive', 
                    loss='squared_loss', 
                    max_iter=1000,
                    n_iter_no_change=42, 
                    penalty='l2', 
                    power_t=0.25, 
                    random_state=69,
                    shuffle=True, 
                    tol=0.001, 
                    validation_fraction=0.1, 
                    verbose=0,
                    warm_start=False)

pipe = Pipeline([('vect', tfidf), 
                 ('scaler', scaler),
                 ('clf', sgdc)])
# Fit Pipeline
pipe.fit(final_df['tokens'].astype(str), final_df['useful'])

# Declaring some parameters
parameters = {
    'clf__n_iter_no_change' : (45, 50),

}

# grid Search
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)

# Find the best fits
grid_search.fit(final_df['tokens'].astype(str), final_df['useful'])

# Best accuracy score via grid search
print(grid_search.best_params_, grid_search.best_score_)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:   33.2s finished


{'clf__n_iter_no_change': 45} 0.01220866577361746


In [71]:
useful_pred = grid_search.predict(final_df['tokens'].astype(str))

In [72]:
print(useful_pred[420])
final_df.iloc[420]

4.377634139321624


useful                                                    5
funny                                                     5
cool                                                      5
text      This place is very unimpressive I've had bette...
tokens    ['place', 'unimpressive', 'better', 'tasting',...
Name: 420, dtype: object

# SGDRegressor Model: 'Funny' Prediction

In [77]:
tfidf = TfidfVectorizer(stop_words='english')
scaler = StandardScaler(with_mean=False, with_std=False)
sgdc = SGDRegressor(alpha=0.0001, 
                    average=False, 
                    early_stopping=False,
                    epsilon=0.1,
                    eta0=0.01, 
                    fit_intercept=True, 
                    l1_ratio=0.15,
                    learning_rate='adaptive', 
                    loss='squared_loss', 
                    max_iter=1000,
                    n_iter_no_change=42, 
                    penalty='l2', 
                    power_t=0.25, 
                    random_state=69,
                    shuffle=True, 
                    tol=0.001, 
                    validation_fraction=0.1, 
                    verbose=0,
                    warm_start=False)

pipe = Pipeline([('vect', tfidf), 
                 ('scaler', scaler),
                 ('clf', sgdc)])
# Fit Pipeline
pipe.fit(final_df['tokens'].astype(str), final_df['cool'])

# Declaring some parameters
parameters = {
    'clf__n_iter_no_change' : (45, 50),

}

# grid Search
grid_search = GridSearchCV(pipe,parameters, cv=15, n_jobs=-1, verbose=1)

# Find the best fits
grid_search.fit(final_df['tokens'].astype(str), final_df['cool'])

# Best accuracy score via grid search
print(grid_search.best_params_, grid_search.best_score_)

Fitting 15 folds for each of 2 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.6min finished


{'clf__n_iter_no_change': 50} -0.0033569464219035746


In [78]:
cool_pred = grid_search.predict(final_df['tokens'].astype(str))

In [79]:
print(cool_pred[400])
final_df.iloc[400]

3.4867875128636214


useful                                                    9
funny                                                     5
cool                                                      5
text      Over the weekend my friend and I had intense c...
tokens    ['weekend', 'friend', 'intense', 'cravings', '...
Name: 400, dtype: object

# SGD Regressor: 'Cool' Prediction

In [80]:
tfidf = TfidfVectorizer(stop_words='english')
scaler = StandardScaler(with_mean=False, with_std=False)
sgdc = SGDRegressor(alpha=0.0001, 
                    average=False, 
                    early_stopping=False,
                    epsilon=0.1,
                    eta0=0.01, 
                    fit_intercept=True, 
                    l1_ratio=0.15,
                    learning_rate='adaptive', 
                    loss='squared_loss', 
                    max_iter=1000,
                    n_iter_no_change=42, 
                    penalty='l2', 
                    power_t=0.25, 
                    random_state=69,
                    shuffle=True, 
                    tol=0.001, 
                    validation_fraction=0.1, 
                    verbose=0,
                    warm_start=False)

pipe = Pipeline([('vect', tfidf), 
                 ('scaler', scaler),
                 ('clf', sgdc)])
# Fit Pipeline
pipe.fit(final_df['tokens'].astype(str), final_df['funny'])

# Declaring some parameters
parameters = {
    'clf__n_iter_no_change' : (45, 50),

}

# grid Search
grid_search = GridSearchCV(pipe,parameters, cv=15, n_jobs=-1, verbose=1)

# Find the best fits
grid_search.fit(final_df['tokens'].astype(str), final_df['funny'])

# Best accuracy score via grid search
print(grid_search.best_params_, grid_search.best_score_)

Fitting 15 folds for each of 2 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  1.7min finished


{'clf__n_iter_no_change': 45} -0.007132759014738393


In [81]:
funny_pred = grid_search.predict(final_df['tokens'].astype(str))

In [82]:
print(funny_pred[3000])
final_df.iloc[3000]

3.414489005549756


useful                                                    6
funny                                                     3
cool                                                      3
text      What happened to Roy's?? Their butterfish used...
tokens    ['happened', 'butterfish', 'defrosted', 'slice...
Name: 3000, dtype: object